In [1]:
import os

import numpy as np
import pandas as pd
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
import plotly.express as px

## Setting up the TPUs

This line is necessary in order to initialize the TPUs. 

Here, "replicas" simply means number of "cores". In the case of GPUs or CPUs, the number of replicas will be 1. [Read this](https://cloud.google.com/tpu/docs/tpus#replicas) for more information.

In [2]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print('Running on TPU ', tpu.master())
except ValueError:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


## Define variables

Make sure to keep those variables in mind as you navigate this notebook! They are all placed below so you can easily change and rerun this notebook.

Don't worry about the model right now. We will come back to it later.

In [3]:
model_name = 'jplu/tf-xlm-roberta-large'
n_epochs = 10
max_len = 80

# Our batch size will depend on number of replicas
batch_size = 16 * strategy.num_replicas_in_sync

## Load datasets

Just regular CSV files. Nothing scary here!

In [4]:
train = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/train.csv')
test = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/test.csv')
submission = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/sample_submission.csv')

## Encode Training data

Now, we need to encode the training and test data into `tokens`, which are numerical representation of our words. To learn more, [read this](https://huggingface.co/transformers/main_classes/tokenizer.html).

In [5]:
# First load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/513 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

In [6]:
# Convert the text so that we can feed it to `batch_encode_plus`
train_text = train[['premise', 'hypothesis']].values.tolist()
test_text = test[['premise', 'hypothesis']].values.tolist()

# Now, we use the tokenizer we loaded to encode the text
train_encoded = tokenizer.batch_encode_plus(
    train_text,
    pad_to_max_length=True,
    max_length=max_len
)

test_encoded = tokenizer.batch_encode_plus(
    test_text,
    pad_to_max_length=True,
    max_length=max_len
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Train and validation split happens here:

In [7]:
x_train, x_valid, y_train, y_valid = train_test_split(
    train_encoded['input_ids'], train.label.values, 
    test_size=0.2, random_state=2020
)

x_test = test_encoded['input_ids']

## Convert to tf.data.Dataset

`tf.data.Dataset` is one of many different ways to define the input to our models. Here, it is a good choice since it is easily compatible with TPUs. Read more about it [in this article](https://towardsdatascience.com/how-to-use-dataset-in-tensorflow-c758ef9e4428).

In [8]:
auto = tf.data.experimental.AUTOTUNE

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(batch_size)
    .prefetch(auto)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(batch_size)
    .cache()
    .prefetch(auto)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(batch_size)
)

## Train the model

It's time to teach our lovely XLM-Roberta how to infer natural language. Notice here we are using `strategy.scope()`. We need to load `transformer_encoder` inside this scope in order to tell Tensorflow that we want our model on the TPUs. Otherwise, it will try to load it in your CPU machine!

XLM-Roberta is one of the best models out there for multilingual classification tasks. Essentially, it is a model that was trained on inherently multilingual text, and used methods that helped it become larger, train longer and on more data! Highly recommend you to read [this blog post by the authors](https://ai.facebook.com/blog/-xlm-r-state-of-the-art-cross-lingual-understanding-through-self-supervision/), as well as the [Huggingface docs](https://huggingface.co/transformers/model_doc/xlmroberta.html) on the subject.

In [9]:
with strategy.scope():
    # First load the transformer layer
    transformer_encoder = TFAutoModel.from_pretrained(model_name)

    # This will be the input tokens 
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")

    # Now, we encode the text using the transformers we just loaded
    sequence_output = transformer_encoder(input_ids)[0]

    # Only extract the token used for classification, which is <s>
    cls_token = sequence_output[:, 0, :]

    # Finally, pass it through a 3-way softmax, since there's 3 possible laels
    out = Dense(3, activation='softmax')(cls_token)

    # It's time to build and compile the model
    model = Model(inputs=input_ids, outputs=out)
    model.compile(
        Adam(lr=1e-5), 
        loss='sparse_categorical_crossentropy', 
        metrics=['accuracy']
    )

model.summary()

Downloading:   0%|          | 0.00/3.05G [00:00<?, ?B/s]

Some layers from the model checkpoint at jplu/tf-xlm-roberta-large were not used when initializing TFXLMRobertaModel: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaModel were initialized from the model checkpoint at jplu/tf-xlm-roberta-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 80)]              0         
_________________________________________________________________
tfxlm_roberta_model (TFXLMRo TFBaseModelOutputWithPool 559890432 
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3075      
Total params: 559,893,507
Trainable params: 559,893,507
Non-trainable params: 0
_________________________________________________________________


/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Unhide below to see the exactly training accuracy and loss after each epoch:

In [10]:
n_steps = len(x_train) // batch_size

train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=n_epochs
)

Epoch 1/10
606/606 [==============================] - 409s 623ms/step - loss: 1.2545 - accuracy: 0.3394 - val_loss: 1.1038 - val_accuracy: 0.3428
Epoch 2/10
606/606 [==============================] - 372s 614ms/step - loss: 1.1876 - accuracy: 0.3317 - val_loss: 1.1011 - val_accuracy: 0.3354
Epoch 3/10
606/606 [==============================] - 372s 614ms/step - loss: 1.1515 - accuracy: 0.3277 - val_loss: 1.0977 - val_accuracy: 0.3519
Epoch 4/10
606/606 [==============================] - 372s 614ms/step - loss: 1.1305 - accuracy: 0.3394 - val_loss: 1.1005 - val_accuracy: 0.3519
Epoch 5/10
606/606 [==============================] - 372s 614ms/step - loss: 1.1328 - accuracy: 0.3336 - val_loss: 1.1225 - val_accuracy: 0.3519
Epoch 6/10
606/606 [==============================] - 372s 614ms/step - loss: 1.1267 - accuracy: 0.3304 - val_loss: 1.1078 - val_accuracy: 0.3354
Epoch 7/10
606/606 [==============================] - 372s 614ms/step - loss: 1.1201 - accuracy: 0.3313 - val_loss: 1.1077 -

In [11]:
#model.save("model.h5")

## Predict on test set and submit

In [12]:
test_preds = model.predict(test_dataset, verbose=1)
submission['prediction'] = test_preds.argmax(axis=1)

325/325 [==============================] - 67s 189ms/step


In [13]:
submission.to_csv('submission.csv', index=False)
submission.head()

,id,prediction
0,c6d58c3f69,0
1,cefcc82292,0
2,e98005252c,0
3,58518c10ba,0
4,c32b0d16df,0


## Visualize Training History

With Plotly Express, this can be done in one function call:

In [14]:
hist = train_history.history

In [15]:
px.line(
    hist, x=range(1, len(hist['loss'])+1), y=['accuracy', 'val_accuracy'], 
    title='Model Accuracy', labels={'x': 'Epoch', 'value': 'Accuracy'}
)

In [16]:
px.line(
    hist, x=range(1, len(hist['loss'])+1), y=['loss', 'val_loss'], 
    title='Model Loss', labels={'x': 'Epoch', 'value': 'Loss'}
)